In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5e09  GCE       4                                       RUNNING  us-central1-a


In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 10 14:28 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'hodisan_ha_melech' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  
  #lines = spark.read.parquet(text)
  word_counter = Counter(tokens)
  our_list = []
  for i in tokens:
    if i not in all_stopwords:
        #if i not in our_list: ### kinda usless...but ok for now
          #occurrences = tokens.count(i)
          if (i,(id,word_counter.get(i))) not in our_list:
            #counter = Counter(tokens)
            #print(counter.get(i))
            our_list.append((i,(id,word_counter.get(i))))
  #print(our_list)
  return our_list
  # YOUR CODE HERE
  #raise NotImplementedError()

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  #print(unsorted_pl)
  sort = sorted(unsorted_pl, key=lambda x:x[0] , reverse=False)

  #end = len(unsorted_pl)  
  #for o in range(0, end):           
  #    for p in range(0, end-o-1):  
  #        if (unsorted_pl[p][-1] > unsorted_pl[p + 1][-1]):  
  #            new_temp = unsorted_pl[p]  
  #            unsorted_pl[p]= unsorted_pl[p + 1]
  #            unsorted_pl[p + 1]= new_temp  
  #return unsorted_pl 
  return sort

  #raise NotImplementedError()
    
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  dic = postings.mapValues(lambda count: len(count))
  return dic
  #raise NotImplementedError()
    
def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''

  dic = postings.map(lambda x: (token2bucket_id(x[0]),x))
  dic = dic.groupByKey()
  dic = dic.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))
  #van = dic.collect()

  #nam = postings.flatMap(lambda x :(token2bucket_id(x[0]),{x}))
  #nam = nam.map(InvertedIndex.write_a_posting_list)
  #ven = nam.collect()
  #print(ven[1])
  #print(ven[0])

  #print(van[0])
  return dic

  # YOUR CODE HERE
  #raise NotImplementedError()

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)

In [ ]:
doc_pairs = parquetFile.select("title", "id").rdd

In [ ]:
# time the index creation time
t_start = time()
# word counts map
folder = "title"
word_counts = doc_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, folder).collect()
index_const_time = time() - t_start
print(index_const_time)

42.02500033378601


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://title_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][953.5 KiB/953.5 KiB]                                                
Operation completed over 1 objects/953.5 KiB.                                    


In [ ]:
print(list(super_posting_locs.items())[:20])

[('bounty', [('0_000.bin', 0)]), ('trentino', [('0_000.bin', 732)]), ('tully', [('0_000.bin', 1356)]), ('reformation', [('0_000.bin', 2160)]), ('herschel', [('0_000.bin', 2640)]), ('papilio', [('0_000.bin', 3300)]), ('amore', [('0_000.bin', 4704)]), ('cecilia', [('0_000.bin', 5154)]), ('profile', [('0_000.bin', 7848)]), ('susan', [('0_000.bin', 8772)]), ('168', [('0_000.bin', 20670)]), ('komnenos', [('0_000.bin', 21078)]), ('tehsil', [('0_000.bin', 21402)]), ('irani', [('0_000.bin', 23526)]), ('retirement', [('0_000.bin', 23868)]), ('georgy', [('0_000.bin', 24618)]), ('democrats', [('0_000.bin', 25752)]), ('election', [('0_000.bin', 27780)]), ('tornado', [('0_000.bin', 209778)]), ('billy', [('0_000.bin', 213288)])]


In [ ]:
index2 = inverted.read_index('.' , 'title_index')

In [ ]:
print(list(index2.df.items())[:20])

[('field', 6350), ('sans', 171), ('east', 10448), ('bar', 1566), ('herrera', 489), ('engine', 1568), ('glass', 1244), ('cuba', 757), ('renfrewshire', 74), ('collapse', 178), ('parry', 358), ('evaristo', 58), ('evolutionary', 146), ('1983', 3192), ('barnes', 1024), ('composer', 1362), ('duck', 595), ('carr', 771), ('demon', 493), ('heath', 1002)]


In [ ]:
!gsutil ls -lh $index_dst

953.54 KiB  2022-01-10T18:45:03Z  gs://hodisan_ha_melech/title/title_index.pkl
TOTAL: 1 objects, 976427 bytes (953.54 KiB)


In [ ]:
doc_pairs = parquetFile.select("anchor_text").rdd
parquetFile.count()

6348910

In [ ]:
doc_pairs = doc_pairs.flatMap(lambda x: x)
doc_pairs = doc_pairs.flatMap(lambda x: x)


In [ ]:
# time the index creation time
t_start = time()
# word counts map
folder = "anchor"
word_counts = doc_pairs.flatMap(lambda x: word_count(x[1], x[0]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, folder).collect()
index_const_time = time() - t_start
print(index_const_time)

924.664320230484


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/{folder}/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 objects/9.5 MiB.                                      


In [ ]:
!gsutil ls -lh $index_dst

  9.46 MiB  2022-01-10T19:01:12Z  gs://hodisan_ha_melech/anchor/anchor_index.pkl
TOTAL: 1 objects, 9917522 bytes (9.46 MiB)


In [ ]:
index2 = inverted.read_index('.' , 'anchor_index')

In [ ]:
print(list(super_posting_locs.items())[:20])

[('argentino', [('0_000.bin', 0)]), ('andronikos', [('0_000.bin', 55092)]), ('beaubassin', [('0_000.bin', 61602)]), ('chocolat', [('0_000.bin', 62064)]), ('quackery', [('0_000.bin', 64356)]), ('paedophilia', [('0_000.bin', 66012)]), ('weichselian', [('0_000.bin', 66654)]), ('kwamina', [('0_000.bin', 67368)]), ('ricci', [('0_000.bin', 67692)]), ('selemani', [('0_000.bin', 86292)]), ('montgomery', [('0_000.bin', 86724)]), ('lubycza', [('0_000.bin', 268566)]), ('pomerelia', [('0_000.bin', 269076)]), ('ornea', [('0_000.bin', 270906)]), ('szilard', [('0_000.bin', 271860)]), ('zubak', [('0_000.bin', 272634)]), ('piri', [('0_000.bin', 273300)]), ('woolly', [('0_000.bin', 276138)]), ('eun-bi', [('0_000.bin', 283812)]), ('vocalise', [('0_000.bin', 284208)])]
